# Slang Generation Demo

This demo will illustrate the basic workflow of the slang generation code accompanying the TACL paper *[A Computational Framework for Slang Generation](https://direct.mit.edu/tacl/article/doi/10.1162/tacl_a_00378/100687/A-Computational-Framework-for-Slang-Generation)* using slang definition data from Urban Dictionary (UD) and conventional definition data from WordNet.

To run this tutorial, you will need the following dependencies:

- Python 3
- Numpy
- Scipy
- tqdm
- NLTK
- Gensim
- PyTorch (torch)
- SBERT (sentence_transformers)
- [CatGO](https://github.com/zhewei-sun/CatGO)


In [1]:
import numpy as np
import torch

We first create a symbolic link pointing to the library. You will need to change the destination if your code sits in a different directory. 

In [2]:
! ln -s ../Code slanggen

CatGO is a library that optimizes and runs models of categorization and can be obtained [here](https://github.com/zhewei-sun/CatGO). Once you have downloaded the code, please link it by replacing the target directory of the simlink below. 

In [3]:
! ln -s ../../CatGO CatGO

In [5]:
from slanggen.util import *
from slanggen.dataloader import WN_Dataset, Urban_Dataset
from slanggen.encoder import FTEncoder, FTCachedEncoder
from slanggen.contrastive import SlangGenTrainer
from slanggen.model import SlangGenModel

Specify a PyTorch device if necessary:

In [6]:
torch.cuda.set_device(0)

Load conventional definition data using the builtin dataloaders. The *.npy* file loaded below contains a pre-processed version of WordNet definition sentences for all words that appear in both WordNet and UD.

In [8]:
wn_data = WN_Dataset('wordnet_urban.npy')
print(wn_data)

Dataset Name: 
Total Definition Entries: 9759
Vocab Size: 1464



Load slang definition data. The *.npy* file loaded below is a pre-processed version of the data released in this repository.

In [9]:
urban_wn_data = Urban_Dataset('Urban_data.npy', wn_data)
print(urban_wn_data)

Dataset Name: 
Total Definition Entries: 2580
Vocab Size: 1436



If you wish to use your own dataset, please create a dataloader object inheriting either *ConvDataset* or *SlangDataset* abstract classes found in *dataloader.py* and following the example data specifications in *dataloader.WN_Dataset* and *dataloader.Urban_Dataset*.

Now let's create a directory to store our results and load in some pre-generated data indices for train-test split:

In [11]:
create_directory('Results')

In [13]:
out_dir='Results/'

dataset = urban_wn_data
slang_inds = DataIndex(np.load('train_ind.npy'), np.load('dev_ind.npy'), np.load('test_ind.npy'))

The following encoder objects initializes a fastText encoder used for collaborative filtering. *FTEncoder* can be used to read in the original fastText embedding file. For efficiency, we have cached the words we need and use a cached encoder instead.

ft_encoder = FTEncoder('path to crawl-300d-2M-subword.vec')

In [15]:
ft_encoder = FTCachedEncoder('ft_embed_cache_Urban.pickle')

The following commands sets up the contrastive trainer and the slang generation model:

In [16]:
trainer = SlangGenTrainer(dataset, word_encoder=ft_encoder, out_dir=out_dir, verbose=True)

In [17]:
model = SlangGenModel(trainer, data_dir=out_dir)
params = {'embed_name':'SBERT_contrastive', 'out_name':'predictions', 'model':'cf_prototype_5', 'prior':None, 'prior_name':'uniform', 'contr_params':None}

Invoke *model.train_contrastive* to train the contrastively learned sense embedding model:

In [18]:
model.train_contrastive(slang_inds, fold_name='urban_wn', params=params)

Generating contrative pairs...


100%|██████████| 129/129 [00:36<00:00,  3.52it/s]


Complete!
Generating triplet data for contrastive training...
Sampled 107614 Triplets
Sampled 6297 Triplets
Complete!


Iteration:   0%|          | 0/6726 [00:00<?, ?it/s]/u/zheweisun/src/Python-3.7.0/lib/python3.7/site-packages/pytorch_transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)

Iteration:   2%|▏         | 119/6726 [00:21<19:43,  5.58it/s]


Iteration:   4%|▎         | 245/6726 [00:44<22:08,  4.88it/s]


...

Iteration:  97%|█████████▋| 6502/6726 [20:17<00:47,  4.75it/s]


Iteration:  99%|█████████▊| 6626/6726 [20:39<00:16,  6.03it/s]


Iteration: 100%|██████████| 6726/6726 [20:56<00:00,  5.35it/s]

Evaluating:  17%|█▋        | 67/394 [00:03<00:14, 21.89it/s]


Epoch: 100%|██████████| 4/4 [1:26:10<00:00, 1292.61s/it]


Encoding sense definitions...
Complete!


Once the contrastive represention has been trained. We run categorization models to perform few-shot learning:

In [21]:
model.train_categorization(slang_inds, fold_name='urban_wn', params=params)

Pre-processing Distances...
Pre-processing Complete!
Optimizing Kernels...


100%|██████████| 1/1 [02:22<00:00, 142.63s/it]


In [22]:
results = model.get_results(fold_name='urban_wn', mode='train', params=params)

We can compute the training AUC as follows. You should have a high AUC of >0.95.

In [23]:
N_train_dev = dataset.N_total - slang_inds.test.shape[0]
train_rankings = get_rankings(results, np.arange(N_train_dev), dataset.vocab_ids[np.concatenate(slang_inds)])
np.mean(get_roc(train_rankings, dataset.V))

0.9576811198082904

Perform prediction on the test portion of our dataset:

In [24]:
model.predict_testset(slang_inds, fold_name='urban_wn', params=params)

Pre-processing Distances...
Pre-processing Complete!
Optimizing Kernels...


100%|██████████| 1/1 [00:00<00:00,  3.60it/s]


In [25]:
results = model.get_results(fold_name='urban_wn', mode='test', params=params)

The test set AUC is computed as below. You should be able to get above >0.75 on the test set.

In [26]:
N_train_dev = dataset.N_total - slang_inds.test.shape[0]
test_rankings = get_rankings(results, np.arange(N_train_dev, dataset.N_total), dataset.vocab_ids[np.concatenate(slang_inds)])
np.mean(get_roc(test_rankings, dataset.V))

0.7618369449704111

Instead of passing all test examples during training. We can also pass in definition sentences and labels post-training as follows:

In [27]:
def collect_slang_sents(dataset, ind):
    sentences = []
    for i in ind:
        sentences.append(' '.join(simple_preprocess(dataset.slang_data[i].def_sent)))
    return sentences

In [28]:
test_sents = collect_slang_sents(dataset, slang_inds.test)
test_labels = dataset.vocab_ids[slang_inds.test]

In [29]:
model.predict_from_definitions(test_sents, test_labels, fold_name='urban_wn', mode='testdef', params=params)

Pre-processing Distances...
Pre-processing Complete!
Optimizing Kernels...


100%|██████████| 1/1 [00:00<00:00,  3.54it/s]


In [30]:
results = model.get_results(fold_name='urban_wn', mode='testdef', params=params)

The test set AUC should be the same as expected:

In [31]:
N_train_dev = dataset.N_total - slang_inds.test.shape[0]
test_rankings = get_rankings(results, np.arange(N_train_dev, dataset.N_total), dataset.vocab_ids[np.concatenate(slang_inds)])
np.mean(get_roc(test_rankings, dataset.V))

0.7618369449704111